In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
# 국내주식 보유기간(STK_HLD_.CSV)
train_df = pd.read_csv('train_1005최최종.csv', encoding='cp949')
test_df = pd.read_csv("test_final_1008.csv", encoding='cp949')

In [3]:
train = train_df.copy()
test = test_df.copy()

In [5]:
len(train)

681472

In [6]:
train.head(2)

,계좌 ID,종목코드,매수일자,보유기간(일),국제유가_매수,기준금리_매수,Rank,매도일자,매수_년,투자성향,...,시장구분,주거래업종_매수,cus_cluster_3,환율 변화율,past_id,기준일자,잔고수량,잔고금액,주당 액면가,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,69.12,1.50,75.0,2018-08-10,2018,3,...,1,0.0,13.0,-0.012353,30.340426,2018-07-26,274.0,11782000.0,5000.0,43000.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2020-01-16,2,57.89,1.25,97.0,2020-01-20,2020,3,...,1,0.0,12.0,0.004671,30.340426,2020-01-16,208.0,6229600.0,5000.0,29950.0


In [7]:
test.head(2)

,계좌 ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),국제유가_매수,기준금리_매수,Rank,투자성향,...,시가총액 규모유형,시장구분,주거래업종_매수,cus_cluster_3,매수_년,기준일자,잔고수량,잔고금액,주당 액면가,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,2020-05-22,153,IDX00001,0,33.8,0.75,38.0,3,...,1,1,0.0,14.0,2020,2020-05-22,300.0,3945000.0,5000.0,13150.0
1,4932d54890232be3d59b064a348e9dc510044d5526b990...,A032640,2020-05-06,165,IDX00484,0,23.7,0.75,40.0,3,...,1,1,0.0,4.0,2020,2020-05-06,500.0,6350000.0,5000.0,12700.0


In [38]:
train.loc[train['cus_cluster_3']==1,'가중치']= 1
train.loc[train['cus_cluster_3']==2,'가중치']= 1
train.loc[train['cus_cluster_3']==3,'가중치']= 1
train.loc[train['cus_cluster_3']==4,'가중치']= 1
train.loc[train['cus_cluster_3']==5,'가중치']= 1

train.loc[train['cus_cluster_3']==6,'가중치']= 1
train.loc[train['cus_cluster_3']==7,'가중치']= 1
train.loc[train['cus_cluster_3']==8,'가중치']= 1
train.loc[train['cus_cluster_3']==9,'가중치']= 1
train.loc[train['cus_cluster_3']==10,'가중치']= 1

train.loc[train['cus_cluster_3']==11,'가중치']= 1
train.loc[train['cus_cluster_3']==12,'가중치']= 1
train.loc[train['cus_cluster_3']==13,'가중치']= 1
train.loc[train['cus_cluster_3']==14,'가중치']= 1
train.loc[train['cus_cluster_3']==15,'가중치']= 1

In [32]:
train.columns

Index(['계좌 ID', '종목코드', '매수일자', '보유기간(일)', '국제유가_매수', '기준금리_매수', 'Rank',
       '매도일자', '매수_년', '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '환율 변화율', 'past_id', '기준일자',
       '잔고수량', '잔고금액', '주당 액면가', 'stk_p', '가중치'],
      dtype='object')

In [33]:
test.columns

Index(['Unnamed: 0', '과거보유일', '국제유가_매수', '기준금리_매수', 'Rank', '투자성향', '서비스.등급',
       '총.투자기간', '시가총액.규모유형', '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년',
       '잔고금액', 'stk_p', '주당.액면가', 'stk_p.1', 'past_id_y'],
      dtype='object')

In [39]:
train['과거보유일'] = train['가중치'] * train['보유기간(일)']

In [40]:
from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from tqdm import tqdm
from sklearn.metrics import (roc_curve, auc, accuracy_score)

In [41]:
train.drop(['가중치'], axis=1, inplace=True)

In [45]:
X = np.array(train.drop(['보유기간(일)',"계좌 ID", "종목코드", "매수일자", "기준일자",'종목한글명','매도일자'], axis=1))
y = train['보유기간(일)'].values

In [46]:
params = {
    'application': 'binary', # for binary classification
#     'num_class' : 1, # used for multi-classes
    'boosting': 'gbdt', # traditional gradient boosting decision tree
    'num_iterations': 100, 
    'learning_rate': 0.05,
    'num_leaves': 62,
    'device': 'cpu', # you can use GPU to achieve faster learning
    'max_depth': -1, # <0 means no limit
    'max_bin': 510, # Small number of bins may reduce training accuracy but can deal with over-fitting
    'lambda_l1': 5, # L1 regularization
    'lambda_l2': 10, # L2 regularization
    'metric' : 'binary_error',
    'subsample_for_bin': 200, # number of samples for constructing bins
    'subsample': 1, # subsample ratio of the training instance
    'colsample_bytree': 0.8, # subsample ratio of columns when constructing the tree
    'min_split_gain': 0.5, # minimum loss reduction required to make further partition on a leaf node of the tree
    'min_child_weight': 1, # minimum sum of instance weight (hessian) needed in a leaf
    'min_child_samples': 5# minimum number of data needed in a leaf
}
# Initiate classifier to use
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt', 
          objective = 'binary', 
          n_jobs = 5, 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'])

# To view the default model parameters:
mdl.get_params().keys()

dict_keys(['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type', 'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight', 'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'silent', 'subsample', 'subsample_for_bin', 'subsample_freq', 'max_bin'])

In [47]:
gridParams = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

grid = GridSearchCV(mdl, gridParams, verbose=1, cv=4, n_jobs=-1)
# Run the grid
grid.fit(X, y)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

Fitting 4 folds for each of 3456 candidates, totalling 13824 fits


/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGSEGV(-11)}

# catboost

In [66]:
train = train_df.copy()
test = test_df.copy()

In [67]:
iem= pd.read_csv('iem_info_20210902.csv')
iem.columns= ["종목코드","종목한글명","종목업종", "시가총액 규모유형","시장구분"]

In [68]:
train.loc[train['cus_cluster_3']==1,'가중치']= 1
train.loc[train['cus_cluster_3']==2,'가중치']= 0.6
train.loc[train['cus_cluster_3']==3,'가중치']= 0.6
train.loc[train['cus_cluster_3']==4,'가중치']= 0.7
train.loc[train['cus_cluster_3']==5,'가중치']= 0.6

train.loc[train['cus_cluster_3']==6,'가중치']= 1
train.loc[train['cus_cluster_3']==7,'가중치']= 0.6
train.loc[train['cus_cluster_3']==8,'가중치']= 0.6
train.loc[train['cus_cluster_3']==9,'가중치']= 0.7
train.loc[train['cus_cluster_3']==10,'가중치']= 0.6

train.loc[train['cus_cluster_3']==11,'가중치']= 1
train.loc[train['cus_cluster_3']==12,'가중치']= 0.6
train.loc[train['cus_cluster_3']==13,'가중치']= 0.6
train.loc[train['cus_cluster_3']==14,'가중치']= 0.7
train.loc[train['cus_cluster_3']==15,'가중치']= 0.6

In [69]:
train['과거 보유일'] = train['가중치'] * train['보유기간(일)']

In [70]:
from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.metrics import (roc_curve, auc, accuracy_score)

In [71]:
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"])
train["종목한글명"] = L_encoder.transform(train["종목한글명"])
test["종목한글명"] = L_encoder.transform(test["종목한글명"])

In [60]:
X = train.drop(['보유기간(일)','가중치',"계좌 ID", "종목코드", "매수일자", "기준일자",'매도일자','환율 변화율'], axis=1)
y = train['보유기간(일)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 42)

In [399]:
def print_best_params(model, params):
    grid_model = GridSearchCV(
        model, 
        param_grid = params,
        cv=10,
        scoring='neg_mean_squared_error')

    grid_model.fit(X_train, y_train)
    rmse = np.sqrt(-1*grid_model.best_score_)
    print(
        '{0} 10 CV 시 최적 평균 RMSE 값 {1}, 최적 alpha:{2}'.format(model.__class__.__name__, np.round(rmse, 4), grid_model.best_params_))
    return grid_model.best_estimator_

In [253]:
cat_params = {
   'depth': [10], 
    'learning_rate' : [0.01],
    'iterations':[100]
}

model_cat  = CatBoostRegressor()
cat_estimator = print_best_params(model_cat, cat_params)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 52.3473611	total: 37.5ms	remaining: 3.71s
1:	learn: 51.8380109	total: 67.3ms	remaining: 3.3s
2:	learn: 51.3352746	total: 97.6ms	remaining: 3.15s
3:	learn: 50.8354526	total: 129ms	remaining: 3.1s
4:	learn: 50.3432075	total: 160ms	remaining: 3.05s
5:	learn: 49.8594888	total: 192ms	remaining: 3s
6:	learn: 49.3742847	total: 222ms	remaining: 2.95s
7:	learn: 48.8949663	total: 261ms	remaining: 3s
8:	learn: 48.4244411	total: 303ms	remaining: 3.06s
9:	learn: 47.9564426	total: 345ms	remaining: 3.1s
10:	learn: 47.4948911	total: 383ms	remaining: 3.1s
11:	learn: 47.0325556	total: 424ms	remaining: 3.11s
12:	learn: 46.5770182	total: 467ms	remaining: 3.13s
13:	learn: 46.1245108	total: 498ms	remaining: 3.06s
14:	learn: 45.6768063	total: 531ms	remaining: 3.01s
15:	learn: 45.2323327	total: 562ms	remaining: 2.95s
16:	learn: 44.7937757	total: 593ms	remaining: 2.9s
17:	learn: 44.3628920	total: 625ms	remaining: 2.85s
18:	learn: 43.9360051	total: 657ms	remaining: 2.8s
19:	learn: 43.5115209	total: 69

In [72]:
test.columns=['계좌 ID', '종목코드', '매수일자', '과거 보유일', '제출ID', '보유기간(일)', '국제유가_매수',
       '기준금리_매수', 'Rank', '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년', '기준일자', '잔고수량', '잔고금액',
       '주당 액면가', 'stk_p', 'past_id']

In [ ]:
from imblearn.over_sampling import SMOTE

# 검증 데이터나 테스트 데이터가 아닌 학습데이터에서만 오버샘플링 사용할 것
smote = SMOTE(random_state=11,k_neighbors=2)
X_train_over, y_train_over = smote.fit_sample(train_data, train_label)

In [402]:
train.columns

Index(['계좌 ID', '종목코드', '매수일자', '국제유가_매수', '기준금리_매수', 'Rank', '매도일자', '매수_년',
       '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형', '시장구분', '주거래업종_매수',
       'cus_cluster_3', '환율 변화율', 'past_id', '기준일자', '잔고수량', '잔고금액', '주당 액면가',
       'stk_p', '가중치', '과거 보유일'],
      dtype='object')

In [10]:
train_label = train["보유기간(일)"]
train.drop(["보유기간(일)"], axis = 1, inplace = True)

In [61]:
train_data = train.drop(["계좌 ID", "종목코드", "매수일자", "기준일자",'환율 변화율','매도일자','가중치'], axis = 1)

In [83]:
df=pd.DataFrame(train['보유기간(일)'].value_counts())

In [106]:
a= df[df['보유기간(일)']==1].index
a

Int64Index([1018, 1131,  803,  792,  791,  790, 1173, 1003,  779,  776,  772,
             661,  771,  653,  805,  962,  959, 1002,  820,  831,  835,  836,
             767,  662,  870,  879,  881,  891,  929,  625,  948,  843,  701,
             652,  713,  722,  720,  693,  716,  715,  695,  697,  688,  709,
             707, 1062,  705,  704,  703, 1140,  686,  763,  745,  762,  757,
             702,  751,  749,  747,  743,  685,  742, 1184,  734,  731,  681,
             683,  672],
           dtype='int64')

In [123]:
train[(train['보유기간(일)']== 1018) |(train['보유기간(일)']== 1131)| (train['보유기간(일)']== 803)|(train['보유기간(일)']== 792)|(train['보유기간(일)']== 791)].index

Int64Index([385, 96358, 500440, 617310, 671442], dtype='int64')

In [124]:
train[(train['보유기간(일)']== 790) |(train['보유기간(일)']== 1173)| (train['보유기간(일)']== 1003)|(train['보유기간(일)']== 779)|(train['보유기간(일)']== 776)].index

Int64Index([92283, 101982, 632070, 644636, 668926], dtype='int64')

In [125]:
train[(train['보유기간(일)']== 772) |(train['보유기간(일)']== 661)| (train['보유기간(일)']== 771)|(train['보유기간(일)']== 653)|(train['보유기간(일)']== 805)].index

Int64Index([5884, 89732, 102011, 492812, 505372], dtype='int64')

In [126]:
train[(train['보유기간(일)']== 962) |(train['보유기간(일)']== 959)| (train['보유기간(일)']== 1002)|(train['보유기간(일)']== 820)|(train['보유기간(일)']== 831)].index

Int64Index([287142, 349029, 560684, 657568, 672295], dtype='int64')

In [127]:
train[(train['보유기간(일)']== 835) |(train['보유기간(일)']== 836)].index

Int64Index([378448, 545617], dtype='int64')

In [131]:
df_train = train.drop(index=[385, 96358, 500440, 617310, 671442,92283, 101982, 632070, 644636, 668926,5884, 89732, 102011, 492812, 505372,
                            287142, 349029, 560684, 657568, 672295,378448, 545617,43216, 109804, 348962,109655, 377955, 496804,544308, 
                             602166, 674023,38011, 657996, 676865,147529, 378861, 673209,505345, 633950, 646465,109299, 378299, 668922,
                            349039, 469064, 658813, 670177, 674040,102031, 348998, 500637, 666155, 680114,162223, 291667, 505400, 595387, 673223,
                            143139, 284173, 417493, 654985, 673236,285491, 676794], 
                      axis=0)

In [132]:
df_train

,계좌 ID,종목코드,매수일자,보유기간(일),국제유가_매수,기준금리_매수,Rank,매도일자,매수_년,투자성향,...,cus_cluster_3,환율 변화율,past_id,기준일자,잔고수량,잔고금액,주당 액면가,stk_p,가중치,과거 보유일
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,69.12,1.50,75.0,2018-08-10,2018,3,...,13.0,-0.012353,30.340426,2018-07-26,274.0,11782000.0,5000.0,43000.0,0.6,6.6
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2020-01-16,2,57.89,1.25,97.0,2020-01-20,2020,3,...,12.0,0.004671,30.340426,2020-01-16,208.0,6229600.0,5000.0,29950.0,0.6,1.2
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,80,64.73,1.50,1.0,2018-05-31,2018,3,...,14.0,0.010379,30.340426,2018-01-31,2.0,4990000.0,5000.0,2495000.0,0.7,56.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,5,71.44,1.50,579.0,2018-05-25,2018,3,...,13.0,0.007625,30.340426,2018-05-17,786.0,14619600.0,1000.0,18600.0,0.6,3.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A036200,2018-05-17,5,71.44,1.50,597.0,2018-05-25,2018,3,...,13.0,0.007625,30.340426,2018-05-17,283.0,2640390.0,500.0,9330.0,0.6,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,dd7b24b8175152c2c6d4b805c8958bcb392d916eb50bcf...,A122230,2017-11-09,25,56.89,1.25,2784.0,2017-12-14,2017,4,...,4.0,-0.019650,9.666667,2017-11-09,2000.0,1000000.0,500.0,500.0,0.7,17.5
681468,dd7b24b8175152c2c6d4b805c8958bcb392d916eb50bcf...,A122230,2018-04-16,1,67.32,1.50,2784.0,2018-04-17,2018,4,...,1.0,0.001273,9.666667,2018-04-16,7500.0,3750000.0,500.0,500.0,1.0,1.0
681469,8c066d434150f388d5a7220c8300e8b43d4086b31e1f0c...,A277290,2017-11-15,133,55.91,1.25,2784.0,2018-06-01,2017,4,...,15.0,-0.035486,133.000000,2017-11-15,1000.0,500000.0,500.0,500.0,0.6,79.8
681470,5b03aed77c1556b6030fd190e501aa8aa6dd0b1f4d28a0...,A285880,2020-07-15,18,40.27,0.50,2784.0,2020-08-10,2020,3,...,4.0,-0.012329,18.000000,2020-07-15,250.0,125000.0,500.0,500.0,0.7,12.6


In [133]:
train_label = df_train["보유기간(일)"]

In [134]:
X = df_train.drop(['보유기간(일)','가중치',"계좌 ID", "종목코드", "매수일자", "기준일자",'매도일자','환율 변화율'], axis=1)
y = df_train['보유기간(일)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 42)

In [137]:
X_test

,국제유가_매수,기준금리_매수,Rank,매수_년,투자성향,서비스 등급,총 투자기간,종목한글명,시가총액 규모유형,시장구분,주거래업종_매수,cus_cluster_3,past_id,잔고수량,잔고금액,주당 액면가,stk_p,과거 보유일
119384,52.11,1.75,732.0,2019,4,5,5,2400,3,99,0.0,7.0,14.017673,100.0,1040000.0,500.0,10400.0,1.2
71491,39.72,0.50,188.0,2020,1,5,2,2422,2,99,0.0,6.0,25.615385,40.0,1080000.0,1000.0,27000.0,1.0
328504,37.37,0.50,1188.0,2020,4,3,2,2037,3,99,0.0,14.0,6.672131,100.0,1140000.0,500.0,11400.0,11.2
295637,22.80,0.75,301.0,2020,2,9,3,826,2,2,0.0,11.0,13.375415,300.0,9030000.0,500.0,30100.0,1.0
54718,41.82,0.50,198.0,2020,4,4,3,1111,2,1,0.0,12.0,8.662441,1.0,17450.0,500.0,17450.0,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28598,25.32,0.75,412.0,2020,5,5,4,2382,2,99,0.0,9.0,62.513908,3.0,57600.0,500.0,19200.0,13.3
501652,56.55,1.50,59.0,2018,5,4,3,1590,1,99,0.0,11.0,2.608696,2.0,138200.0,500.0,69100.0,1.0
343914,33.08,0.75,366.0,2020,5,3,4,2223,3,99,0.0,4.0,38.340824,11.0,48345.0,1000.0,4395.0,46.2
226134,47.46,1.25,789.0,2020,4,5,3,1246,2,99,1.0,11.0,9.597360,20.0,136800.0,500.0,6840.0,1.0


In [142]:
from imblearn.over_sampling import RandomOverSampler
# define oversampling strategy
oversample = RandomOverSampler(random_state=42)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_test, y_test)

In [150]:
X_over.shape

(24663510, 18)

In [149]:
X_over.drop_duplicates()

,국제유가_매수,기준금리_매수,Rank,매수_년,투자성향,서비스 등급,총 투자기간,종목한글명,시가총액 규모유형,시장구분,주거래업종_매수,cus_cluster_3,past_id,잔고수량,잔고금액,주당 액면가,stk_p,과거 보유일
0,52.11,1.75,732.0,2019,4,5,5,2400,3,99,0.0,7.0,14.017673,100.0,1040000.0,500.0,10400.0,1.2
1,39.72,0.50,188.0,2020,1,5,2,2422,2,99,0.0,6.0,25.615385,40.0,1080000.0,1000.0,27000.0,1.0
2,37.37,0.50,1188.0,2020,4,3,2,2037,3,99,0.0,14.0,6.672131,100.0,1140000.0,500.0,11400.0,11.2
3,22.80,0.75,301.0,2020,2,9,3,826,2,2,0.0,11.0,13.375415,300.0,9030000.0,500.0,30100.0,1.0
4,41.82,0.50,198.0,2020,4,4,3,1111,2,1,0.0,12.0,8.662441,1.0,17450.0,500.0,17450.0,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136277,25.32,0.75,412.0,2020,5,5,4,2382,2,99,0.0,9.0,62.513908,3.0,57600.0,500.0,19200.0,13.3
136278,56.55,1.50,59.0,2018,5,4,3,1590,1,99,0.0,11.0,2.608696,2.0,138200.0,500.0,69100.0,1.0
136279,33.08,0.75,366.0,2020,5,3,4,2223,3,99,0.0,4.0,38.340824,11.0,48345.0,1000.0,4395.0,46.2
136280,47.46,1.25,789.0,2020,4,5,3,1246,2,99,1.0,11.0,9.597360,20.0,136800.0,500.0,6840.0,1.0


In [147]:
pd.set_option('display.max_rows', 1000)

In [151]:
X_over

,국제유가_매수,기준금리_매수,Rank,매수_년,투자성향,서비스 등급,총 투자기간,종목한글명,시가총액 규모유형,시장구분,주거래업종_매수,cus_cluster_3,past_id,잔고수량,잔고금액,주당 액면가,stk_p,과거 보유일
0,52.11,1.75,732.0,2019,4,5,5,2400,3,99,0.0,7.0,14.017673,100.0,1040000.0,500.0,10400.0,1.2
1,39.72,0.50,188.0,2020,1,5,2,2422,2,99,0.0,6.0,25.615385,40.0,1080000.0,1000.0,27000.0,1.0
2,37.37,0.50,1188.0,2020,4,3,2,2037,3,99,0.0,14.0,6.672131,100.0,1140000.0,500.0,11400.0,11.2
3,22.80,0.75,301.0,2020,2,9,3,826,2,2,0.0,11.0,13.375415,300.0,9030000.0,500.0,30100.0,1.0
4,41.82,0.50,198.0,2020,4,4,3,1111,2,1,0.0,12.0,8.662441,1.0,17450.0,500.0,17450.0,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24663505,49.08,1.25,802.0,2017,3,4,4,1816,3,99,0.0,10.0,786.000000,15533.0,39065495.0,200.0,2515.0,471.6
24663506,49.08,1.25,802.0,2017,3,4,4,1816,3,99,0.0,10.0,786.000000,15533.0,39065495.0,200.0,2515.0,471.6
24663507,49.08,1.25,802.0,2017,3,4,4,1816,3,99,0.0,10.0,786.000000,15533.0,39065495.0,200.0,2515.0,471.6
24663508,49.08,1.25,802.0,2017,3,4,4,1816,3,99,0.0,10.0,786.000000,15533.0,39065495.0,200.0,2515.0,471.6


In [136]:
from imblearn.over_sampling import SMOTE

# 검증 데이터나 테스트 데이터가 아닌 학습데이터에서만 오버샘플링 사용할 것
smote = SMOTE(random_state=11,k_neighbors=2)
X_train_over, y_train_over = smote.fit_sample(X_test, y_test)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3

In [12]:
test.columns=['계좌 ID', '종목코드', '매수일자', '과거 보유일', '제출ID', '보유기간(일)', '국제유가_매수',
       '기준금리_매수', 'Rank', '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년', '기준일자', '잔고수량', '잔고금액',
       '주당 액면가', 'stk_p', 'past_id']

In [13]:
test_data = test.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자",], axis = 1)

In [532]:
train_data.columns

Index(['국제유가_매수', '기준금리_매수', 'Rank', '매수_년', '투자성향', '서비스 등급', '총 투자기간',
       '종목한글명', '시가총액 규모유형', '시장구분', '주거래업종_매수', 'cus_cluster_3', 'past_id',
       '잔고수량', '잔고금액', '주당 액면가', 'stk_p', '과거 보유일'],
      dtype='object')

In [533]:
test_data.columns

Index(['과거 보유일', '국제유가_매수', '기준금리_매수', 'Rank', '투자성향', '서비스 등급', '총 투자기간',
       '종목한글명', '시가총액 규모유형', '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년',
       '잔고수량', '잔고금액', '주당 액면가', 'stk_p', 'past_id'],
      dtype='object')

In [534]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=100,
                          learning_rate=0.1,
                          depth=10)
# Fit model
model.fit(train_data, train_label)

# Get predictions
preds = model.predict(test_data)

0:	learn: 47.5298767	total: 56ms	remaining: 5.55s
1:	learn: 42.9622197	total: 112ms	remaining: 5.48s
2:	learn: 38.8042691	total: 164ms	remaining: 5.31s
3:	learn: 35.0280402	total: 214ms	remaining: 5.14s
4:	learn: 31.6410152	total: 264ms	remaining: 5.02s
5:	learn: 28.5851687	total: 313ms	remaining: 4.9s
6:	learn: 25.8154126	total: 360ms	remaining: 4.79s
7:	learn: 23.3291512	total: 416ms	remaining: 4.79s
8:	learn: 21.0831755	total: 465ms	remaining: 4.7s
9:	learn: 19.0493267	total: 512ms	remaining: 4.61s
10:	learn: 17.2118439	total: 558ms	remaining: 4.51s
11:	learn: 15.5584072	total: 610ms	remaining: 4.47s
12:	learn: 14.0647712	total: 657ms	remaining: 4.4s
13:	learn: 12.7243533	total: 704ms	remaining: 4.33s
14:	learn: 11.5145872	total: 752ms	remaining: 4.26s
15:	learn: 10.4181749	total: 800ms	remaining: 4.2s
16:	learn: 9.4318740	total: 849ms	remaining: 4.14s
17:	learn: 8.5484280	total: 896ms	remaining: 4.08s
18:	learn: 7.7482702	total: 945ms	remaining: 4.03s
19:	learn: 7.0329264	total: 99

In [535]:
preds

array([253.10077089, 261.7152104 , 690.26689318, ..., 720.14709069,
       527.80341172, 689.70708816])

In [536]:
submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [537]:
submission["hold_d"] = np.round(preds)

In [538]:
submission.columns=["제출ID","보유기간(일)"]

In [539]:
sub = pd.merge(submission, test[['제출ID','과거 보유일']], on=['제출ID'])
sub['차이']= sub['보유기간(일)']-sub['과거 보유일']

In [540]:
# 0~146인 값만 채택, 나머지는 outlier 취급 ㅎ
def change(x):
        if x< 0 : return 0
        elif x>146 : return 146
        else: return x

In [545]:
from pandas import Series
# outlier는 완전 146으로 하고, 나머지는 0~146으로 스케일링 조절해서 0~146 사이에 오게 만들어주기!
a= sub['차이'].quantile(0.1)
b= sub['차이'].quantile(0.95)
def min_max(x):
    if x <= a : return 1
    elif x >= b : return 146
    else : return (x-a)/(b-a)*146

In [546]:
sub["보정1"] = sub["차이"].apply(change)
sub["보정2"] = round(sub["차이"].apply(min_max))
sub['hold_d'] = sub['보정2']+sub['과거 보유일']
sub =sub.rename(columns={'제출ID':'submit_id'})
submission = sub[['submit_id','hold_d']]

In [547]:
sub

,submit_id,보유기간(일),과거 보유일,차이,보정1,보정2,hold_d
0,IDX00001,253.0,153,100.0,100.0,53.0,206.0
1,IDX00002,262.0,335,-73.0,0.0,23.0,358.0
2,IDX00003,690.0,139,551.0,146.0,132.0,271.0
3,IDX00004,211.0,236,-25.0,0.0,31.0,267.0
4,IDX00005,32.0,9,23.0,23.0,40.0,49.0
...,...,...,...,...,...,...,...
70591,IDX70592,757.0,23,734.0,146.0,146.0,169.0
70592,IDX70593,702.0,103,599.0,146.0,141.0,244.0
70593,IDX70594,720.0,858,-138.0,0.0,11.0,869.0
70594,IDX70595,528.0,11,517.0,146.0,126.0,137.0


In [544]:
submission.to_csv("sub30.csv", header=True, index=False, encoding='cp949')

In [16]:
model_CBR = CatBoostRegressor()
parameters = {'depth': [8,10,12], 
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations':[50, 100,150]
             }

In [91]:
grid = GridSearchCV(estimator=model_CBR, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

0:	learn: 48.6260129	total: 601ms	remaining: 1m 29s
1:	learn: 45.4663358	total: 855ms	remaining: 1m 3s
2:	learn: 42.4013596	total: 1.11s	remaining: 54.2s
3:	learn: 39.7212831	total: 1.36s	remaining: 49.8s
4:	learn: 37.1206167	total: 1.63s	remaining: 47.2s
5:	learn: 35.1922841	total: 2.07s	remaining: 49.6s
6:	learn: 33.3783773	total: 2.48s	remaining: 50.7s
7:	learn: 31.7687717	total: 2.8s	remaining: 49.8s
8:	learn: 30.2883348	total: 3.2s	remaining: 50.2s
9:	learn: 29.1704971	total: 3.61s	remaining: 50.5s
10:	learn: 28.1809513	total: 3.84s	remaining: 48.6s
11:	learn: 27.2265499	total: 4.04s	remaining: 46.4s
12:	learn: 26.4136342	total: 4.23s	remaining: 44.6s
13:	learn: 25.5970815	total: 4.42s	remaining: 43s
14:	learn: 24.8385788	total: 4.61s	remaining: 41.5s
15:	learn: 24.3763796	total: 4.81s	remaining: 40.3s
16:	learn: 23.7323293	total: 5s	remaining: 39.2s
17:	learn: 23.1755076	total: 5.19s	remaining: 38.1s
18:	learn: 22.8066190	total: 5.38s	remaining: 37.1s
19:	learn: 22.5547951	total:

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fd200890e10>,
             n_jobs=-1,
             param_grid={'depth': [8, 10, 12], 'iterations': [50, 100, 150],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [93]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.8596026349237516

 The best parameters across ALL searched params:
 {'depth': 12, 'iterations': 150, 'learning_rate': 0.1}


In [98]:
train.columns

Index(['보유기간(일)', '국제유가_매수', '기준금리_매수', 'Rank', '매수_년', '투자성향', '서비스 등급',
       '총 투자기간', '시가총액 규모유형', '시장구분', '주거래업종_매수', 'cus_cluster_3', '환율 변화율',
       'past_id', '잔고수량', '잔고금액', '주당 액면가', 'stk_p'],
      dtype='object')

In [99]:
test.columns

Index(['계좌 ID', '종목코드', '매수일자', '과거 보유일', '제출ID', '보유기간(일)', '국제유가_매수',
       '기준금리_매수', 'Rank', '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년', '기준일자', '잔고수량', '잔고금액',
       '주당 액면가', 'stk_p'],
      dtype='object')

In [17]:
train_data = train.drop(['환율 변화율','past_id'], axis=1) 
test_data = test.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis=1)

In [18]:
train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [19]:
train_data.columns

Index(['국제유가_매수', '기준금리_매수', 'Rank', '매수_년', '투자성향', '서비스 등급', '총 투자기간',
       '종목한글명', '시가총액 규모유형', '시장구분', '주거래업종_매수', 'cus_cluster_3', '잔고수량',
       '잔고금액', '주당 액면가', 'stk_p'],
      dtype='object')

In [20]:
test_data.columns

Index(['과거 보유일', '국제유가_매수', '기준금리_매수', 'Rank', '투자성향', '서비스 등급', '총 투자기간',
       '종목한글명', '시가총액 규모유형', '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년',
       '잔고수량', '잔고금액', '주당 액면가', 'stk_p'],
      dtype='object')

In [117]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

# evaluate the model
model = CatBoostRegressor(depth = 12, iterations = 150, learning_rate=0.1, n_estimators = 200)

models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = CatBoostRegressor(depth = 12, 
                              iterations = 150, 
                              learning_rate=0.1, 
                              n_estimators = 200)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric =["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

TypeError: fit() got an unexpected keyword argument 'eval_metric'

In [191]:
# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [29]:
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]  

In [36]:
# fit the model on the whole dataset
from sklearn.metrics import accuracy_score

model = CatBoostRegressor(depth = 12, iterations = 150, learning_rate=0.1, loss_function='RMSE')

# train the model
model.fit(
     X_train, y_train,
     eval_set=(X_test, y_test), eval_metric=["rmse"],
    cat_features=categorical_features_indices,
    verbose=300,
)

TypeError: fit() got an unexpected keyword argument 'eval_metric'

In [31]:
y_pred = model.predict(X_test)

In [32]:
y_pred

array([ 1.25894513,  2.29966514,  1.60562979, ...,  5.02959192,
       10.14144954,  1.94657602])

In [ ]:
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

# LGBM

In [422]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

In [475]:
train = train_df.copy()
test = test_df.copy()

In [476]:
test.columns=['계좌 ID', '종목코드', '매수일자', '과거 보유일', '제출ID', '보유기간(일)', '국제유가_매수',
       '기준금리_매수', 'Rank', '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년', '기준일자', '잔고수량', '잔고금액',
       '주당 액면가', 'stk_p', 'past_id']

In [477]:
iem= pd.read_csv('iem_info_20210902.csv')
iem.columns= ["종목코드","종목한글명","종목업종", "시가총액 규모유형","시장구분"]

In [478]:
train.loc[train['cus_cluster_3']==1,'가중치']= 1
train.loc[train['cus_cluster_3']==2,'가중치']= 0.6
train.loc[train['cus_cluster_3']==3,'가중치']= 0.7
train.loc[train['cus_cluster_3']==4,'가중치']= 0.7
train.loc[train['cus_cluster_3']==5,'가중치']= 0.6

train.loc[train['cus_cluster_3']==6,'가중치']= 1
train.loc[train['cus_cluster_3']==7,'가중치']= 0.6
train.loc[train['cus_cluster_3']==8,'가중치']= 0.7
train.loc[train['cus_cluster_3']==9,'가중치']= 0.7
train.loc[train['cus_cluster_3']==10,'가중치']= 0.6

train.loc[train['cus_cluster_3']==11,'가중치']= 1
train.loc[train['cus_cluster_3']==12,'가중치']= 0.6
train.loc[train['cus_cluster_3']==13,'가중치']= 0.7
train.loc[train['cus_cluster_3']==14,'가중치']= 0.7
train.loc[train['cus_cluster_3']==15,'가중치']= 0.6

In [479]:
train['과거보유일'] = train['가중치'] * train['보유기간(일)']

In [480]:
train.columns

Index(['계좌 ID', '종목코드', '매수일자', '보유기간(일)', '국제유가_매수', '기준금리_매수', 'Rank',
       '매도일자', '매수_년', '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '환율 변화율', 'past_id', '기준일자',
       '잔고수량', '잔고금액', '주당 액면가', 'stk_p', '가중치', '과거보유일'],
      dtype='object')

In [481]:
test.columns

Index(['계좌 ID', '종목코드', '매수일자', '과거 보유일', '제출ID', '보유기간(일)', '국제유가_매수',
       '기준금리_매수', 'Rank', '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년', '기준일자', '잔고수량', '잔고금액',
       '주당 액면가', 'stk_p', 'past_id'],
      dtype='object')

In [482]:
from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"])
train["종목한글명"] = L_encoder.transform(train["종목한글명"])
test["종목한글명"] = L_encoder.transform(test["종목한글명"])

In [483]:
X = train.drop(['보유기간(일)','가중치',"계좌 ID", "종목코드", "매수일자", "기준일자",'매도일자','환율 변화율','Rank','시장구분','잔고수량', '잔고금액','stk_p'], axis=1)
y = train['보유기간(일)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 42)

In [455]:
def print_best_params(model, params):
    grid_model = GridSearchCV(
        model, 
        param_grid = params,
        cv=10,
        scoring='neg_mean_squared_error')

    grid_model.fit(X_train, y_train)
    rmse = np.sqrt(-1*grid_model.best_score_)
    print(
        '{0} 10 CV 시 최적 평균 RMSE 값 {1}, 최적 alpha:{2}'.format(model.__class__.__name__, np.round(rmse, 4), grid_model.best_params_))
    return grid_model.best_estimator_

In [456]:
lgb_params = {
    'objective':['regression'],
    'num_leave' : [1],
    'learning_rate' : [0.05],
    'n_estimators':[1000],
    'max_bin' : [80],
    'gpu_id':[0] ,         
    'tree_method':['gpu_hist'],
    'predictor':['gpu_predictor'],
    'refit':[True]
}

lgb_model = LGBMRegressor()
lgb_estimator = print_best_params(lgb_model, lgb_params)

/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1224: UserWarning: predictor keyword has been found in `params` and will be ignored.
Please use predictor argument of the Dataset constructor to pass this parameter.
  .format(key))


[LightGBM] [Warning] Unknown parameter: refit
[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] Unknown parameter: gpu_id
[LightGBM] [Warning] Unknown parameter: num_leave
[LightGBM] [Warning] Unknown parameter: tree_method


/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1224: UserWarning: predictor keyword has been found in `params` and will be ignored.
Please use predictor argument of the Dataset constructor to pass this parameter.
  .format(key))


[LightGBM] [Warning] Unknown parameter: refit
[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] Unknown parameter: gpu_id
[LightGBM] [Warning] Unknown parameter: num_leave
[LightGBM] [Warning] Unknown parameter: tree_method


/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1224: UserWarning: predictor keyword has been found in `params` and will be ignored.
Please use predictor argument of the Dataset constructor to pass this parameter.
  .format(key))


...

[LightGBM] [Warning] Unknown parameter: refit
[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] Unknown parameter: gpu_id
[LightGBM] [Warning] Unknown parameter: num_leave
[LightGBM] [Warning] Unknown parameter: tree_method


/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1224: UserWarning: predictor keyword has been found in `params` and will be ignored.
Please use predictor argument of the Dataset constructor to pass this parameter.
  .format(key))


[LightGBM] [Warning] Unknown parameter: refit
[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] Unknown parameter: gpu_id
[LightGBM] [Warning] Unknown parameter: num_leave
[LightGBM] [Warning] Unknown parameter: tree_method


/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1224: UserWarning: predictor keyword has been found in `params` and will be ignored.
Please use predictor argument of the Dataset constructor to pass this parameter.
  .format(key))


[LightGBM] [Warning] Unknown parameter: refit
[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] Unknown parameter: gpu_id
[LightGBM] [Warning] Unknown parameter: num_leave
[LightGBM] [Warning] Unknown parameter: tree_method
LGBMRegressor 10 CV 시 최적 평균 RMSE 값 3.2687, 최적 alpha:{'gpu_id': 0, 'learning_rate': 0.05, 'max_bin': 80, 'n_estimators': 1000, 'num_leave': 1, 'objective': 'regression', 'predictor': 'gpu_predictor', 'refit': True, 'tree_method': 'gpu_hist'}


In [438]:
test.columns

Index(['계좌.ID', '종목코드', '매수일자', '과거.보유일', '제출ID', '보유기간.일.', '국제유가_매수',
       '기준금리_매수', 'Rank', '투자성향', '서비스.등급', '총.투자기간', '종목한글명', '시가총액.규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년', '기준일자', '잔고수량', '잔고금액',
       '주당.액면가', 'stk_p', 'past_id'],
      dtype='object')

In [439]:
X_train.columns

Index(['국제유가_매수', '기준금리_매수', 'Rank', '매수_년', '투자성향', '서비스 등급', '총 투자기간',
       '종목한글명', '시가총액 규모유형', '시장구분', '주거래업종_매수', 'cus_cluster_3', 'past_id',
       '잔고수량', '잔고금액', '주당 액면가', 'stk_p', '과거보유일'],
      dtype='object')

In [457]:
test_data = test.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자",'Rank','시장구분','잔고수량', '잔고금액','stk_p'], axis = 1)

In [458]:
lgb_preds = lgb_estimator.predict(test_data)

In [459]:
lgb_preds

array([ 49.0023545 ,  77.07822385, 537.74675761, ..., 193.73611121,
       126.23513378, 395.84354122])

In [460]:
submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [461]:
submission["hold_d"] = np.round(lgb_preds)

In [462]:
submission.columns=["제출ID","보유기간(일)"]

In [463]:
sub = pd.merge(submission, test[['제출ID','과거 보유일']], on=['제출ID'])
sub['차이']= sub['보유기간(일)']-sub['과거 보유일']

In [464]:
# 0~146인 값만 채택, 나머지는 outlier 취급 ㅎ
def change(x):
        if x< 0 : return 0
        elif x>146 : return 146
        else: return x

In [516]:
from pandas import Series
# outlier는 완전 146으로 하고, 나머지는 0~146으로 스케일링 조절해서 0~146 사이에 오게 만들어주기!
a= sub['차이'].quantile(0.1)
b= sub['차이'].quantile(0.95)
def min_max(x):
    if x < a : return 1
    elif x > b : return 146
    else : return (x-a)/(b-a)*146

In [513]:
sub['차이'].quantile(0.95)

176.0

In [517]:
sub["보정1"] = sub["차이"].apply(change)
sub["보정2"] = round(sub["차이"].apply(min_max))
sub['hold_d'] = sub['보정2']+sub['과거 보유일']
sub =sub.rename(columns={'제출ID':'submit_id'})
submission = sub[['submit_id','hold_d']]

In [518]:
sub

,submit_id,보유기간(일),과거 보유일,차이,보정1,보정2,hold_d
0,IDX00001,49.0,153,-104.0,0.0,49.0,202.0
1,IDX00002,77.0,335,-258.0,0.0,1.0,336.0
2,IDX00003,538.0,139,399.0,146.0,146.0,285.0
3,IDX00004,58.0,236,-178.0,0.0,24.0,260.0
4,IDX00005,102.0,9,93.0,93.0,117.0,126.0
...,...,...,...,...,...,...,...
70591,IDX70592,72.0,23,49.0,49.0,102.0,125.0
70592,IDX70593,273.0,103,170.0,146.0,144.0,247.0
70593,IDX70594,194.0,858,-664.0,0.0,1.0,859.0
70594,IDX70595,126.0,11,115.0,115.0,125.0,136.0


In [519]:
submission.to_csv("sub29.csv", header=True, index=False, encoding='cp949')

In [358]:
train['계좌 ID'].nunique()

9337

In [354]:
test['계좌 ID'].nunique()

9980

In [359]:
train_id = pd.DataFrame(train['계좌 ID'].unique())
test_id = pd.DataFrame(train['계좌 ID'].unique())

In [374]:
df_id = pd.concat([train_id, test_id], ignore_index=True)
df_id

,0
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...
1,1c52768eebea2eab6a7ed8cc95d838a3732fc6dce51ba3...
2,77c0b012b37b39c0e67a13deb9110620914d23a90288d8...
3,6012ca0aa8039c4c282e0e63bcd6b223bcef0692883b90...
4,77e7d5de53556b330c1ec036e4f50d89023d68dde93888...
...,...
18669,13e879cd0546d5fe05325436e3a604f3dfad25b3fd1c7c...
18670,dd7b24b8175152c2c6d4b805c8958bcb392d916eb50bcf...
18671,8c066d434150f388d5a7220c8300e8b43d4086b31e1f0c...
18672,5b03aed77c1556b6030fd190e501aa8aa6dd0b1f4d28a0...


In [375]:
df_id.drop_duplicates()

,0
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...
1,1c52768eebea2eab6a7ed8cc95d838a3732fc6dce51ba3...
2,77c0b012b37b39c0e67a13deb9110620914d23a90288d8...
3,6012ca0aa8039c4c282e0e63bcd6b223bcef0692883b90...
4,77e7d5de53556b330c1ec036e4f50d89023d68dde93888...
...,...
9332,13e879cd0546d5fe05325436e3a604f3dfad25b3fd1c7c...
9333,dd7b24b8175152c2c6d4b805c8958bcb392d916eb50bcf...
9334,8c066d434150f388d5a7220c8300e8b43d4086b31e1f0c...
9335,5b03aed77c1556b6030fd190e501aa8aa6dd0b1f4d28a0...


In [376]:
df_id.isnull().sum()

0    0
dtype: int64

In [492]:
X_train.drop(['주거래업종_매수'],axis=1, inplace=True)

/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [496]:
X_train.drop(['투자성향'],axis=1, inplace=True)

/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [499]:
X_train.drop(['총 투자기간'],axis=1, inplace=True)

/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [502]:
X_train.drop(['시가총액 규모유형'],axis=1, inplace=True)

/home/yoonjeong.won/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [503]:
X_train

,국제유가_매수,기준금리_매수,매수_년,서비스 등급,종목한글명,cus_cluster_3,past_id,주당 액면가,과거보유일
242601,40.18,0.50,2020,4,2013,3.0,4.825328,500.0,3.5
111484,68.59,1.50,2018,9,455,12.0,12.609598,500.0,1.8
132573,74.42,1.50,2018,9,2428,4.0,15.397351,500.0,34.3
109903,39.73,0.50,2020,4,1657,11.0,3.305412,100.0,1.0
308603,58.85,1.25,2019,3,451,13.0,5.893162,200.0,4.2
...,...,...,...,...,...,...,...,...,...
259178,40.34,0.50,2020,3,423,7.0,14.395833,5000.0,1.2
365838,36.54,0.50,2020,9,1990,13.0,4.675824,500.0,5.6
131932,40.77,0.50,2020,4,3052,8.0,13.098160,100.0,6.3
671155,40.84,0.50,2020,3,1570,13.0,9.473684,500.0,7.0


In [490]:
y_train.shape

(545177,)

In [508]:
from imblearn.over_sampling import SMOTE

# 검증 데이터나 테스트 데이터가 아닌 학습데이터에서만 오버샘플링 사용할 것
smote = SMOTE(random_state=11, k_neighbors=2)
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE, ADASYN
X, y = make_classification(n_classes=3, weights=[0.03, 0.07,0.9],n_features=10, 
                           n_clusters_per_class=1, n_samples=2000, random_state=10)

print('Original dataset shape %s' % Counter(y))
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))
ada=ADASYN(random_state=0)
X_syn,y_syn=ada.fit_resample(X,y)
print('Resampled dataset shape from ADASYN %s' % Counter(y_syn))